In [ ]:

# Import the PyDrive module
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create a PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Get the dataset file from Google Drive
file_id = '1Z-yb752A3o7b9dqrGt24XU0sl53FVqya' # Replace with the file ID of the dataset
downloaded = drive.CreateFile({'id': file_id})

# Download the dataset file to the local Colab instance
downloaded.GetContentFile('trda.csv')

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('trda.csv')

In [ ]:
df["Answer_possible"] = df["Answer_possible"].map({True:False,False:True})

In [ ]:
df = df[df["Answer_possible"]==False]

In [ ]:
df.to_csv('final.csv')

In [ ]:
import csv
import json

# Read the CSV file
data = []
with open("final.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        data.append(row)

# Convert the data to the desired JSON format
json_data ={
    "data": [
        {
            
                    "context": row['Paragraph'],
                    "qas": [
                        {
                            "answers": [
                                {
                                    "text": (row['Answer_text'])[2:-2],
                                    "answer_start": (row['Answer_start'])[1:-1],
                                    
                                }
                            ],
                            "id": row['Unnamed: 0'],
                            "is_impossible": row['Answer_possible'],
                            "question": row['Question'],
                            
                            
                        }
                    ],
                }
                for row in data
            
    ]
}


with open("squad.json", "w") as f:
    json.dump(json_data, f)


In [ ]:
for i in json_data["data"]:
  j = (i["qas"])[0]
  k = j["is_impossible"]
  if k=="False":
    j["is_impossible"]= False
  else:
    j["is_impossible"]= True
  # print((j["answers"])[0])
  
  if j["is_impossible"]==False:
    collect = [{
        "text": str(j['answers'][0]["text"]),
        "answer_start": int(j["answers"][0]["answer_start"]),
    }]
    j['answers']=collect;
  else:
    j['answers']=[]

  

In [ ]:
train = json_data["data"][:10000]
test = json_data["data"][20010:25000]

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import math
import random

import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
     


In [ ]:


from simpletransformers.question_answering import QuestionAnsweringModel


train_args = {
    'learning_rate': 3e-5,
    'num_train_epochs': 1,
    'max_seq_length': 384,
    'doc_stride': 128,
    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    'train_batch_size': 6,
    'gradient_accumulation_steps': 8,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name":"hello"+"hello"},
}

model = QuestionAnsweringModel('albert', 'albert-base-v1', args=train_args,use_cuda=True)
# distilbert-base-cased-distilled-squad


model.train_model(train)
     


Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertForQuestionAnswering: ['predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at albert-base-v1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN t

convert squad examples to features:   0%|          | 0/10000 [00:00<?, ?it/s]Could not find answer: '"This Can't Be Life",' vs. '"This Can\'t Be Life"'
Could not find answer: '"Hey Mama", as well as a cover of Journey's "Don't Stop Believin'", to' vs. '"Hey Mama", as well as a cover of Journey\'s "Don\'t Stop Believin\'"'
Could not find answer: 'John Lennon's "Imagine"' vs. 'John Lennon\'s "Imagine"'
convert squad examples to features:   0%|          | 1/10000 [00:27<77:45:16, 27.99s/it]Could not find answer: '6'2"' vs. '6\'2"'
Could not find answer: '5'10"' vs. '5\'10"'
add example index and unique id: 100%|██████████| 10000/10000 [00:00<00:00, 760305.99it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adarshkln5. Use `wandb login --relogin` to force relogin


Running Epoch 0 of 1:   0%|          | 0/1824 [00:00<?, ?it/s]

(228, 2.2948691213438126)

In [ ]:
for i in test:
  i['qas'][0]['answers']=[]

In [ ]:
model.save_model('./results/')

In [ ]:
import sklearn

In [ ]:
"""Official evaluation script for SQuAD version 2.0.

In addition to basic functionality, we also compute additional statistics and
plot precision-recall curves if an additional na_prob.json file is provided.
This file is expected to map question ID's to the model's predicted probability
that a question is unanswerable.
"""
import argparse
import collections
import json
import numpy as np
import os
import re
import string
import sys

OPTS = None

def parse_args():
  parser = argparse.ArgumentParser('Official evaluation script for SQuAD version 2.0.')
  parser.add_argument('data_file', metavar='data.json', help='Input data JSON file.')
  parser.add_argument('pred_file', metavar='pred.json', help='Model predictions.')
  parser.add_argument('--out-file', '-o', metavar='eval.json',
                      help='Write accuracy metrics to file (default is stdout).')
  parser.add_argument('--na-prob-file', '-n', metavar='na_prob.json',
                      help='Model estimates of probability of no answer.')
  parser.add_argument('--na-prob-thresh', '-t', type=float, default=1.0,
                      help='Predict "" if no-answer probability exceeds this (default = 1.0).')
  parser.add_argument('--out-image-dir', '-p', metavar='out_images', default=None,
                      help='Save precision-recall curves to directory.')
  parser.add_argument('--verbose', '-v', action='store_true')
  if len(sys.argv) == 1:
    parser.print_help()
    sys.exit(1)
  return parser.parse_args()

def make_qid_to_has_ans(dataset):
  qid_to_has_ans = {}
  for article in dataset:
    for p in article['paragraphs']:
      for qa in p['qas']:
        qid_to_has_ans[qa['id']] = bool(qa['answers'])
  return qid_to_has_ans

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
  return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common1 = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = len(common1.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    if gold_toks==pred_toks:
      return 1
    else:
      return 0
    # return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def get_raw_scores(dataset, preds):
  exact_scores = {}
  f1_scores = {}
  for article in dataset:
    for p in article['paragraphs']:
      for qa in p['qas']:
        qid = qa['id']
        gold_answers = [a['text'] for a in qa['answers']
                        if normalize_answer(a['text'])]
        if not gold_answers:
          # For unanswerable questions, only correct answer is empty string
          gold_answers = ['']
        if qid not in preds:
          print('Missing prediction for %s' % qid)
          continue
        a_pred = preds[qid]
        # Take max over all gold answers
        exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
        f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
  return exact_scores, f1_scores

def apply_no_ans_threshold(scores, na_probs, qid_to_has_ans, na_prob_thresh):
  new_scores = {}
  for qid, s in scores.items():
    pred_na = na_probs[qid] > na_prob_thresh
    if pred_na:
      new_scores[qid] = float(not qid_to_has_ans[qid])
    else:
      new_scores[qid] = s
  return new_scores

def make_eval_dict(exact_scores, f1_scores, qid_list=None):
  if not qid_list:
    total = len(exact_scores)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores.values()) / total),
        ('f1', 100.0 * sum(f1_scores.values()) / total),
        ('total', total),
    ])
  else:
    total = len(qid_list)
    return collections.OrderedDict([
        ('exact', 100.0 * sum(exact_scores[k] for k in qid_list) / total),
        ('f1', 100.0 * sum(f1_scores[k] for k in qid_list) / total),
        ('total', total),
    ])

def merge_eval(main_eval, new_eval, prefix):
  for k in new_eval:
    main_eval['%s_%s' % (prefix, k)] = new_eval[k]

def plot_pr_curve(precisions, recalls, out_image, title):
  plt.step(recalls, precisions, color='b', alpha=0.2, where='post')
  plt.fill_between(recalls, precisions, step='post', alpha=0.2, color='b')
  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.xlim([0.0, 1.05])
  plt.ylim([0.0, 1.05])
  plt.title(title)
  plt.savefig(out_image)
  plt.clf()

def make_precision_recall_eval(scores, na_probs, num_true_pos, qid_to_has_ans,
                               out_image=None, title=None):
  qid_list = sorted(na_probs, key=lambda k: na_probs[k])
  true_pos = 0.0
  cur_p = 1.0
  cur_r = 0.0
  precisions = [1.0]
  recalls = [0.0]
  avg_prec = 0.0
  for i, qid in enumerate(qid_list):
    if qid_to_has_ans[qid]:
      true_pos += scores[qid]
    cur_p = true_pos / float(i+1)
    cur_r = true_pos / float(num_true_pos)
    if i == len(qid_list) - 1 or na_probs[qid] != na_probs[qid_list[i+1]]:
      # i.e., if we can put a threshold after this point
      avg_prec += cur_p * (cur_r - recalls[-1])
      precisions.append(cur_p)
      recalls.append(cur_r)
  if out_image:
    plot_pr_curve(precisions, recalls, out_image, title)
  return {'ap': 100.0 * avg_prec}

def run_precision_recall_analysis(main_eval, exact_raw, f1_raw, na_probs, 
                                  qid_to_has_ans, out_image_dir):
  if out_image_dir and not os.path.exists(out_image_dir):
    os.makedirs(out_image_dir)
  num_true_pos = sum(1 for v in qid_to_has_ans.values() if v)
  if num_true_pos == 0:
    return
  pr_exact = make_precision_recall_eval(
      exact_raw, na_probs, num_true_pos, qid_to_has_ans,
      out_image=os.path.join(out_image_dir, 'pr_exact.png'),
      title='Precision-Recall curve for Exact Match score')
  pr_f1 = make_precision_recall_eval(
      f1_raw, na_probs, num_true_pos, qid_to_has_ans,
      out_image=os.path.join(out_image_dir, 'pr_f1.png'),
      title='Precision-Recall curve for F1 score')
  oracle_scores = {k: float(v) for k, v in qid_to_has_ans.items()}
  pr_oracle = make_precision_recall_eval(
      oracle_scores, na_probs, num_true_pos, qid_to_has_ans,
      out_image=os.path.join(out_image_dir, 'pr_oracle.png'),
      title='Oracle Precision-Recall curve (binary task of HasAns vs. NoAns)')
  merge_eval(main_eval, pr_exact, 'pr_exact')
  merge_eval(main_eval, pr_f1, 'pr_f1')
  merge_eval(main_eval, pr_oracle, 'pr_oracle')

def histogram_na_prob(na_probs, qid_list, image_dir, name):
  if not qid_list:
    return
  x = [na_probs[k] for k in qid_list]
  weights = np.ones_like(x) / float(len(x))
  # plt.hist(x, weights=weights, bins=20, range=(0.0, 1.0))
  # plt.xlabel('Model probability of no-answer')
  # plt.ylabel('Proportion of dataset')
  # plt.title('Histogram of no-answer probability: %s' % name)
  # plt.savefig(os.path.join(image_dir, 'na_prob_hist_%s.png' % name))
  # plt.clf()

def find_best_thresh(preds, scores, na_probs, qid_to_has_ans):
  num_no_ans = sum(1 for k in qid_to_has_ans if not qid_to_has_ans[k])
  cur_score = num_no_ans
  best_score = cur_score
  best_thresh = 0.0
  qid_list = sorted(na_probs, key=lambda k: na_probs[k])
  for i, qid in enumerate(qid_list):
    if qid not in scores: continue
    if qid_to_has_ans[qid]:
      diff = scores[qid]
    else:
      if preds[qid]:
        diff = -1
      else:
        diff = 0
    cur_score += diff
    if cur_score > best_score:
      best_score = cur_score
      best_thresh = na_probs[qid]
  return 100.0 * best_score / len(scores), best_thresh

def find_all_best_thresh(main_eval, preds, exact_raw, f1_raw, na_probs, qid_to_has_ans):
  best_exact, exact_thresh = find_best_thresh(preds, exact_raw, na_probs, qid_to_has_ans)
  best_f1, f1_thresh = find_best_thresh(preds, f1_raw, na_probs, qid_to_has_ans)
  main_eval['best_exact'] = best_exact
  main_eval['best_exact_thresh'] = exact_thresh
  main_eval['best_f1'] = best_f1
  main_eval['best_f1_thresh'] = f1_thresh




In [ ]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          12985        4485        1318          13        7181        8193
Swap:             0           0           0


In [ ]:
# sum1 = 0
# count=0
# for i in test[:1000]:
#   count+=1
#   answer = model.predict([i])[0]
#   id = int(answer[0]["id"])
#   act_ans = (df[df["Unnamed: 0"]==id]["Answer_text"]).to_list()[0][2:-2]
#   pred_ans = answer[0]["answer"][0]
#   sum1 = sum1+compute_f1(act_ans,pred_ans)
#   # compute_f1(act_ans,pred_ans)
# print(sum1/count)
  

In [ ]:
test[0]

{'context': 'In 1165 Maimonides visited Jerusalem and prayed on the Temple Mount, in the "great, holy house". In 1141 Spanish-Jewish poet, Yehuda Halevi, issued a call to the Jews to emigrate to the Land of Israel, a journey he undertook himself. In 1187 Sultan Saladin, founder of the Ayyubid dynasty, defeated the Crusaders in the Battle of Hattin and subsequently captured Jerusalem and almost all of Palestine. In time, Saladin issued a proclamation inviting Jews to return and settle in Jerusalem, and according to Judah al-Harizi, they did: "From the day the Arabs took Jerusalem, the Israelites inhabited it." Al-Harizi compared Saladin\'s decree allowing Jews to re-establish themselves in Jerusalem to the one issued by the Persian king Cyrus the Great over 1,600 years earlier.',
 'qas': [{'answers': [{'text': ' ', 'answer_start': 0}],
   'id': '50414',
   'is_impossible': False,
   'question': 'Who was the founder of the Ayyubid dynasty?'}]}

In [ ]:
print("f1-score for 1000 test samples is:",sum1/count)

f1-score for 1000 test samples is: 0.6777806027929832


In [ ]:
def task_do(paras,question,ids):
  json_data ={
      "data": [
          {
              
                      "context": paras[i],
                      "qas": [
                          {
                              "answers": [
                                  
                              ],
                              "id": ids[i],
                              "is_impossible": False,
                              "question": question,
                              
                              
                          }
                      ],
                  }
                  for i in range(len(paras))
              
      ]
  }


  with open("squad.json", "w") as f:
      json.dump(json_data, f)
  json_data = json_data["data"]
  is_possible = 0
  for j in json_data:
    pre = model.predict([j])[0]
    fin_pre = pre[0]["answer"][0]
    if fin_pre!='':
      is_possible = 1
      print(pre[0]['id'])
      print(pre[0]["answer"][0])
      break
  if is_possible==0:
    print('Not possible to answer')




INTEGRATING A RANDOM FOREST MODEL FOR TASK ONE TO REDUCE TRAINING TIME AND LOAD LATENCY ON TASK 2

In [ ]:
fd = pd.read_csv("paragraphs.csv")
fd1 = pd.read_csv('question_answers.csv')

NameError: ignored